In [10]:
import os
import sys

# Add the project root directory to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

# Set the DJANGO_SETTINGS_MODULE environment variable
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'pyserver.settings')

# Initialize Django
import django
django.setup()

# Now you can import Django settings
from django.conf import settings

# Rest of your imports
import pandas as pd
import pickle
from sqlalchemy import create_engine
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

BASE_DIR = settings.BASE_DIR
MODEL_PATH = os.path.join(BASE_DIR, 'model', 'recommender_model.pkl')
RAW_DATA_PATH = os.path.join(BASE_DIR, 'data', 'raw', 'products.csv')

def fetch_data_from_postgres():
    db_settings = settings.DATABASES['default']
    db_url = f"postgresql://{db_settings['USER']}:{db_settings['PASSWORD']}@{db_settings['HOST']}:{db_settings['PORT']}/{db_settings['NAME']}"
    engine = create_engine(db_url)
    
    query = "SELECT * FROM products"
    df = pd.read_sql_query(query, engine)
    if df.empty:
        raise ValueError("No products found in the database")
    return df

def save_data_to_csv(df):
    os.makedirs(os.path.dirname(RAW_DATA_PATH), exist_ok=True)
    df.to_csv(RAW_DATA_PATH, index=False)

def train_model(df):
    df['combined_features'] = df['name'] + ' ' + df['description'] + ' ' + df['brand'] + ' ' + df['country_of_origin'] + ' ' + df['tags']
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df['combined_features'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return df, cosine_sim

def save_model(df, cosine_sim):
    os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)
    with open(MODEL_PATH, 'wb') as f:
        pickle.dump((df, cosine_sim), f)

def update_and_train_product_recommend():
    try:
        df = fetch_data_from_postgres()
        save_data_to_csv(df)
        df, cosine_sim = train_model(df)
        save_model(df, cosine_sim)
        return "Data fetched, model trained and saved successfully"
    except Exception as e:
        return f"Error: {str(e)}"

if __name__ == "__main__":
    result = update_and_train_product_recommend()
    print(result)

ValueError: Invalid truth value: warn